In [51]:
ROOT = r"X:\doc_layout_analysis\signature\test"
classes = ["signature"]

In [52]:
from PIL import Image
import cv2
import os
import matplotlib.pyplot as plt

def invert_image(image_path):
    image = Image.open(image_path)
    inverted_image = Image.eval(image, lambda x: 255 - x)
    return inverted_image


def read_yolo_labels(label_path):
    with open(label_path, 'r') as file:
        lines = file.readlines()
    labels = []
    for line in lines:
        class_id, x_center, y_center, width, height = map(float, line.strip().split())
        labels.append((int(class_id), x_center, y_center, width, height))
    return labels

def visualize_image_with_labels(image_path, labels, classes):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    height, width, _ = image.shape
    print(labels)
    for label in labels:
        class_id, x_center, y_center, w, h = label
        x_center, y_center, w, h = x_center * width, y_center * height, w * width, h * height
        x1, y1, x2, y2 = int(x_center - w / 2), int(y_center - h / 2), int(x_center + w / 2), int(y_center + h / 2)
        cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(image, classes[class_id], (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2) 

    return image

In [53]:
# for root, dirs, files in os.walk(ROOT.replace("train", "test")):
#     for file in files:
#         if file.endswith(('jpg', 'jpeg', 'png')):
#             file_path = os.path.join(root, file)
#             image = Image.open(file_path).convert("L")
#             histogram = image.histogram()
#             black_pixels = histogram[0]
#             total_pixels = sum(histogram)
#             black_ratio = black_pixels / total_pixels

#             # Invert the image if it is mostly black
#             if black_ratio > 0.4:
#                 inverted_image = invert_image(file_path)
#                 inverted_image.save(file_path)

# p = r""
# invert_image(p).save(p)

In [54]:
from collections import defaultdict


def delete_labels_per_class(label_path, c):
    labels = read_yolo_labels(label_path)
    new_labels = [label for label in labels if label[0] != c]
    with open(label_path, 'w') as file:
        for label in new_labels:
            file.write(f"{label[0]} {label[1]} {label[2]} {label[3]} {label[4]}\n")


def check_label_distribution(labellist):
    # Count the number of each label in the files
    label_counts = defaultdict(int)
    for labelfile in labellist:
        with open(labelfile) as f:
            for line in f:
                parts = line.split()
                if parts:
                    label_counts[parts[0]] += 1

    # Print the counts
    print("Label counts:")
    for label, count in label_counts.items():
        print(f"{label}: {count}")

In [55]:
image_dir = os.path.join(ROOT, 'images')
label_dir = os.path.join(ROOT, 'labels')

image_files = [f for f in os.listdir(image_dir) if f.endswith(('jpg', 'png', 'jpeg'))]
label_files = [".".join(f.split('.')[:-1]) + '.txt' for f in image_files]

check_label_distribution([os.path.join(label_dir, f) for f in label_files])

for i, (image_file, label_file) in enumerate(zip(image_files, label_files)):
    image_path = os.path.join(image_dir, image_file)
    label_path = os.path.join(label_dir, label_file)
    delete_labels_per_class(label_path, 1)
    labels = read_yolo_labels(label_path)
    
    # vizimg = visualize_image_with_labels(image_path, labels, classes)
    # plt.imshow(vizimg)
    # plt.show()
    # if i == 10:
    #     break

check_label_distribution([os.path.join(label_dir, f) for f in label_files])

Label counts:
0: 41
1: 5
Label counts:
0: 41
